# Stats

Connect to the mondo database.

In [1]:
import pymongo

mongo_client = pymongo.MongoClient('mongodb://localhost:27017')

mongo_client.drop_database('stats')

print("MongoDB:", mongo_client.server_info)

MongoDB: <bound method MongoClient.server_info of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>


Create a stats database with a collection named `records` that contains the number of documents for each database's collections.

In [2]:
for database in mongo_client.list_database_names():
    if database not in ('admin','config','local','stats'):
        stats = { 'database': database }
        collection_records = {}
        for collection in mongo_client[database].list_collection_names():
            collection_records[collection] = mongo_client[database][collection].count_documents({})
        stats['collections'] = collection_records
        mongo_client['stats']['records'].insert_one(stats)

In the stats database create a new collection named `attributes` that contains all the attributes for each database's collections.

In [3]:
for database in mongo_client.list_database_names():
    if database not in ('admin','config','local','stats'):
        for collection in mongo_client[database].list_collection_names():
            stats = { 
                'database': database,
                'collection': collection
            }
            attributes = mongo_client[database][collection].find_one({}).keys()
            stats['attributes'] = list(attributes)
            stats['attributes'].remove('_id')
            mongo_client['stats']['attributes'].insert_one(stats)